# Azure AI Face-Aware Content Understanding for Video 
## Objective
This document is meant to present a guideline on how to leverage the Azure AI Content Understanding API for your video content with face features enabled.

## Pre-requisites
1. [Azure Account and Subscription](https://azure.microsoft.com/en-us/free/)
2. [Azure AI Resource](https://review.learn.microsoft.com/en-us/azure/ai-studio/how-to/create-azure-ai-resource?branch=main)
3. [Azure AI Resource Face Gating](https://learn.microsoft.com/en-us/legal/cognitive-services/computer-vision/limited-access-identity?context=%2Fazure%2Fai-services%2Fcomputer-vision%2Fcontext%2Fcontext#registration-process) Select `[Video Indexer] Facial Identification (1:N or 1:1 matching) to search for a face in a media or entertainment video archive to find a face within a video and generate metadata for media or entertainment use cases only` in the registration form.

### Install the necessary packages

In [ ]:
%pip install -r ./requirements.txt

## !!! Restart the kernel if any package was installed or updated in the installation step !!!

## Set configurations and review checklist

All configurations are expected to be set in the file `./.env`. If the file does not exist, it will be created. You will need to open the file and check/set all configuration variables.

Verify each item in the checklist below while you are working on the file `./.env`.
1. Make sure you have an Azure account.
1. Make sure you have an Azure AI Services resource that will be used to access Azure Content Understanding
   1. Obtain the endpoint of this resource and assign it to `AZURE_AI_SERVICES_ENDPOINT`.
   1. The pre-filled value for `AZURE_AI_SERVICES_API_VERSION` is good as of August 2024.
   1. For authentication to this resource
      1. If key-based authentication is used, obtain the API key and assign it to `AZURE_AI_SERVICES_API_KEY`.
      1. Otherwise, Entra ID/AAD authentication is used. You will need to
         1. Leave the value for `AZURE_AI_SERVICES_API_KEY` empty.
         2. Open a terminal and run `az login --use-device-code` to login to Azure
         3. Assign `Cognitive Service User` role to yourself for this resource

1. For `VIDEO_LOCATION`, it could be a video URL or a local file.
   1. If you put a video url, make sure your video is publicly accessible.
   1. A local file with the absolute path. If you use this option, the video length must be less than 30 minutes and file size must be less than 500MB

If each item in the checklist is checked, save the file `./.env`.

In [ ]:
import os

if os.path.exists(".env"):
    print(".env file already exists. Assuming all configurations are set.")
else:
    print(".env file does not exist. Creating one.")
    print("Please open the file and check/set all configurations.")
    print("After that, you may proceed to the next cell.")
    !cp .env.example .env

    assert False, "Stop the notebook execution on purpose to allow user configuration." 

### Load Environment Variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env", override=True)

VIDEO_LOCATION = os.getenv("VIDEO_LOCATION", "./inputs/{{VIDEO_FILENAME}}")
AZURE_AI_SERVICES_ENDPOINT = os.getenv("AZURE_AI_SERVICES_ENDPOINT", "")
AZURE_AI_SERVICES_API_VERSION = os.getenv("AZURE_AI_SERVICES_API_VERSION", "")
AZURE_AI_SERVICES_API_KEY = os.getenv("AZURE_AI_SERVICES_API_KEY", "")


In [3]:
assert VIDEO_LOCATION, "VIDEO_LOCATION is not set"
assert AZURE_AI_SERVICES_ENDPOINT, "AZURE AI SERVICES ENDPOINT is not set"

### Create a custom analyzer with pre-defined schema for videos
The custom analyzer schema is defined in [./templates/video_face_aware.json](./templates/video_face_aware.json)

In [ ]:
from pathlib import Path
import sys
import json
import uuid

parent_dir = Path(Path.cwd()).parents[0]

sys.path.append(str(parent_dir))  # add
print(str(parent_dir))
from utility.content_understanding_client import AzureContentUnderstandingClient

ANALYZER_TEMPLATE_PATH = "./templates/video_face_aware.json"
ANALYZER_ID = "video_face_analyzer" + "_" + str(
    uuid.uuid4())  # Unique identifier for the analyzer

# Create the Content Understanding (CU) client
cu_client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_SERVICES_ENDPOINT,
    api_version=AZURE_AI_SERVICES_API_VERSION,
    subscription_key=AZURE_AI_SERVICES_API_KEY,
    enable_face_identification=True)

# Use the client to create an analyzer
response = cu_client.begin_create_analyzer(
    ANALYZER_ID, analyzer_schema_path=ANALYZER_TEMPLATE_PATH)

result = cu_client.poll_result(response)

print(json.dumps(result, indent=2))

### Use the created analyzer to extract video content

In [ ]:
# Submit the video for content analysis
response = cu_client.begin_analyze(ANALYZER_ID, file_location=VIDEO_LOCATION)

# Wait for the analysis to complete and get the content analysis result
video_cu_result = cu_client.poll_result(
    response, timeout_seconds=3600)  # 1 hour timeout

# Print the content analysis result
print(f"Video Content Understanding result: ", video_cu_result)

### Optional: Write Results Out to File

In [ ]:
import json

results_file = "./outputs/video_cu_results.json"
with open(results_file, "w", encoding="utf-8") as f:
    json.dump(video_cu_result, f, indent=4)

### Optional: Get and Save Key Frames and Face Thumbnails

In [ ]:
from PIL import Image
from io import BytesIO
import re


def save_image(image_id: str):
    raw_image = cu_client.get_image_from_analyze_operation(analyze_response=response,
        image_id=image_id
    )
    image = Image.open(BytesIO(raw_image))
    # image.show()
    image.save(f"{image_id}.jpg", "JPEG")


# Initialize sets for unique face IDs and keyframe IDs
face_ids = set()
keyframe_ids = set()

# Extract unique face IDs safely
result_data = video_cu_result.get("result", {})
contents = result_data.get("contents", [])

# Iterate over contents to find faces and keyframes if available
for content in contents:
    # Safely retrieve face IDs if "faces" exists and is a list
    faces = content.get("faces", [])
    if isinstance(faces, list):
        for face in faces:
            face_id = face.get("faceId")
            if face_id:
                face_ids.add(f"face.{face_id}")

    # Extract keyframe IDs from "markdown" if it exists and is a string
    markdown_content = content.get("markdown", "")
    if isinstance(markdown_content, str):
        keyframe_ids.update(re.findall(r"(keyFrame\.\d+)\.jpg", markdown_content))

# Output the results
print("Unique Face IDs:", face_ids)
print("Unique Keyframe IDs:", keyframe_ids)

# Save all face images
for face_id in face_ids:
    save_image(face_id)

# Save all keyframe images
for keyframe_id in keyframe_ids:
    save_image(keyframe_id)

## Delete exist analyzer in Azure Understanding Content Service
This snippet is not required, but it's only used to prevent the testing analyzer from residing in your service. The custom fields analyzer could be stored in your service for reusing by subsequent business in real usage scenarios.

In [ ]:
cu_client.delete_analyzer(ANALYZER_ID)